In [97]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [110]:
#-------------------Extract-------------------------

url = "https://forecast.weather.gov/MapClick.php?lat=41.884250000000065&lon=-87.63244999999995#.Y-q4ltLMJkj"
r = requests.get(url)

soup = BeautifulSoup(r.content,"html.parser")

#various containers
item1 = soup.find_all(id='current_conditions-summary')
item2 = soup.find_all(id='current_conditions_detail')

#raw data
temp_f = [item.find(class_="myforecast-current-lrg").get_text() for item in item1]
temp_c = [item.find(class_="myforecast-current-sm").get_text() for item in item1]

#df of temperatures
df_temp = pd.DataFrame({"Temperature_F" : temp_f, "Temperature_C": temp_c})

#df_2 is a df of current conditions in detail
table = soup.find_all('table')
df_2 = pd.read_html(str(table))[0]
df_2 = df_2.pivot(columns=0, values=1).ffill().dropna().reset_index().drop(columns=['index'])

#merge both dataframes
source_df=pd.concat([df_temp,df_2],axis=1)

#scrape lattitude, longitude, and elevation 
lat_lon_elev = soup.find('span', {'class': 'smallTxt'}).text.strip()
lat, lon, elev = re.findall(r'[-+]?\d*\.\d+|\d+', lat_lon_elev)

#scrape name
location = soup.find('h2', {'class': 'panel-title'}).text.strip()

#add location, lat, long, and elev to source_df
source_df['elevation_ft'] = elev
source_df['latitude'] = lat
source_df['longitude'] = lon
source_df['location'] = location


display(source_df)



,Temperature_F,Temperature_C,Barometer,Dewpoint,Humidity,Last update,Visibility,Wind Chill,Wind Speed,elevation_ft,latitude,longitude,location
0,50°F,10°C,29.87 in (1011.9 mb),18°F (-8°C),28%,13 Feb 4:53 pm CST,10.00 mi,46°F (8°C),W 9 mph,617,41.78,87.76,"Chicago, Chicago Midway Airport (KMDW)"


In [ ]:
#-------------Transform------------------------
